In [9]:
from inaFaceGender import GenderVideo
from inaFaceGender.opencv_utils import get_fps
import os

In [2]:
gv = GenderVideo()

In [164]:
subsamp = 2
ret = gv('./media/pexels-artem-podrez-5725953.mp4', subsamp_coeff=subsamp)

In [165]:
ret

,frame,bb,label,decision,conf
0,1.0,"(61, 100, 274, 313)",f,-7.373473,0.998221
1,1.0,"(950, -6, 1130, 172)",m,12.188662,0.989907
2,1.0,"(744, 17, 933, 206)",f,-8.159047,0.987154
3,1.0,"(558, 199, 733, 375)",m,18.365358,0.953532
4,1.0,"(312, 161, 497, 347)",f,-12.675145,0.786215
...,...,...,...,...,...
618,355.0,"(613, 262, 794, 443)",m,12.529735,0.973860
619,355.0,"(338, 114, 483, 260)",f,-18.028181,0.950689
620,357.0,"(876, 66, 1009, 198)",f,-9.150864,0.996197
621,357.0,"(613, 263, 795, 444)",m,11.919030,0.981238


In [166]:
fps = get_fps('./media/pexels-artem-podrez-5725953.mp4')
fps

30.0

In [167]:
ass_template='''[Script Info]
Title:
Original Script:
Original Translation:
Original Editing:
Original Timing:
Original Script Checking:
ScriptType: v4.00+
Collisions: Normal
PlayResY: 720
PlayResX: 1280

; 720 au lieu de 404
PlayDepth: 0
Timer: 100,0000
[V4+ Styles]
Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding
Style: subtitle,DejaVu Sans,80,&H00FFFFFF,&H00B4FCFC,&H00000000,&H00000000,0,0,0,0,100,100,0.00,0.00,1,2.50,0.00,2,50,50,50,0
Style: subtit25,DejaVu Sans,25,&H00FFFFFF,&H00B4FCFC,&H00000000,&H00000000,0,0,0,0,100,100,0.00,0.00,1,2.50,0.00,2,50,50,50,0
Style: concepts,DejaVu Sans,80,&H0000C000,&H00B4FCFC,&H00000000,&H00000000,0,0,0,0,100,100,0.00,0.00,1,2.50,0.00,7,50,50,50,0
Style: personte,DejaVu Sans,80,&H00C00000,&H00B4FCFC,&H00000000,&H00000000,0,0,0,0,100,100,0.00,0.00,1,2.50,0.00,6,50,50,50,0
Style: personfa,DejaVu Sans,80,&H00C000C0,&H00B4FCFC,&H00000000,&H00000000,0,0,0,0,100,100,0.00,0.00,1,2.50,0.00,6,50,50,50,0
Style: personvo,DejaVu Sans,80,&H00C000C0,&H00B4FCFC,&H00000000,&H00000000,0,0,0,0,100,100,0.00,0.00,1,2.50,0.00,4,50,50,50,0
Style: facebox,DejaVu Sans,25,&HFF000000,&H00B4FCFC,&H000000FF,&H00000000,0,0,0,0,100,100,0.00,0.00,1,2.50,0.00,7,50,50,50,0
Style: link,DejaVu Sans,80,&H000000FF,&H00B4FCFC,&H00000000,&H00000000,0,0,0,0,100,100,0.00,0.00,1,2.50,0.00,4,50,50,50,0
[Events]
Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV,Effect, Text'''


In [168]:
with open('mytest.ass', 'wt') as fid:
    print(ass_template, file=fid)
    for _, e in ret.iterrows():
        start = sec2hmsms(e.frame / fps)
        stop = sec2hmsms((e.frame + subsamp) / fps)
        left, top, right, bottom = e.bb
        shape = 'm %d %d l %d %d %d %d %d %d' % (left, top, right, top, right, bottom, left, bottom)
        text = e.label + '_%.1f' % e.decision
        #text='m8'
        print(text)
        print('Dialogue: 1,%s,%s,facebox,foo,000,000,000,,{\pos(0,0)\p1\\3c&H00FFFF&}%s{\p0\\r}' % (start, stop, shape), file=fid)
        print('Dialogue: 1,%s,%s,subtitle,foo,000,000,000,,{\\an7\pos(%d,%d)}%s' % (start, stop, left, top, text), file=fid)

        #print('Dialogue: 1,0:0:21.40,0:0:21.43,facebox,foo,000,000,000,,{\an7\pos(430,101)\1c&H007F00&}f10{\r}')
        # print('Dialogue: 1,%s,%s,facebox,foo,000,000,000,,{\\an7\pos(%d,%d)\\1c&H00FFFF&}%s{\\r}' % (start, stop, left, top, text), file=fid)
        #print('Dialogue: 1,%s,%s,facebox,foo,000,000,000,,{\\an7\pos(309,86)\\1c&H0000FF&}f0TEST{\\r}' % (start, stop), file=fid)

f_-7.4
m_12.2
f_-8.2
m_18.4
f_-12.7
f_-10.3
m_13.1
f_-7.5
m_19.9
f_-11.9
f_-10.9
m_11.7
f_-7.5
m_19.4
f_-12.8
f_-7.5
m_11.9
f_-7.5
m_18.9
f_-13.0
f_-7.9
m_11.5
f_-6.3
m_19.1
f_-13.4
f_-7.5
m_11.6
f_-5.1
m_18.6
f_-13.9
f_-6.8
f_-2.9
m_12.2
m_17.9
f_-14.2
f_-8.1
f_-3.2
m_12.2
m_17.9
f_-13.1
f_-8.6
m_11.9
f_-3.3
m_17.9
f_-13.0
f_-8.2
m_11.4
f_-4.2
m_16.7
f_-13.5
f_-8.0
m_12.8
f_-5.5
m_17.2
f_-13.9
f_-8.6
m_12.3
f_-6.1
m_17.1
f_-12.7
f_-9.2
m_11.6
f_-5.5
m_16.7
f_-14.0
f_-10.3
m_11.1
f_-5.9
m_16.4
f_-12.8
f_-10.1
m_12.3
f_-4.8
m_16.4
f_-13.7
f_-10.1
m_12.0
f_-5.8
m_15.4
f_-15.5
f_-11.0
m_11.8
f_-5.9
m_15.5
f_-17.2
f_-11.4
m_11.3
f_-6.4
m_15.3
f_-15.9
f_-11.2
m_10.8
f_-5.6
m_15.2
f_-15.9
f_-10.1
m_10.2
f_-6.1
m_15.2
f_-16.6
f_-10.8
m_11.0
f_-6.2
m_15.3
f_-16.9
f_-9.7
m_11.2
f_-5.7
m_17.6
f_-17.8
f_-11.0
m_11.8
f_-6.2
m_17.1
f_-16.2
f_-9.7
m_11.3
f_-6.6
m_17.6
f_-19.4
f_-9.4
m_10.9
f_-6.7
m_18.0
f_-19.3
f_-9.8
m_10.4
f_-7.4
m_17.1
f_-18.7
f_-10.1
f_-8.3
m_9.6
m_17.0
f_-19.4
f_-10.5
m_10.5
f_

In [169]:
!vlc ./media/pexels-artem-podrez-5725953.mp4 --sub-file ./mytest.ass
#!vlc ./media/pexels-artem-podrez-5725953.mp4 --sub-file ./media/000000-face-ina-1.ass

VLC media player 3.0.9.2 Vetinari (revision 3.0.9.2-0-gd4c1aefe4d)
[0000563674554670] main libvlc: Running vlc with the default interface. Use 'cvlc' to use vlc without interface.
[ass] Shaper: FriBidi 0.19.7 (SIMPLE) HarfBuzz-ng 2.6.4 (COMPLEX)
[ass] Using font provider fontconfig
[00007f2978c15b20] avcodec decoder: Using NVIDIA VDPAU Driver Shared Library  460.73.01  Thu Apr  1 21:30:56 UTC 2021 for hardware decoding
[ass] fontselect: (DejaVu Sans, 400, 0) -> /usr/share/fonts/truetype/dejavu/DejaVuSans.ttf, 0, DejaVuSans
[ass] Shaper: FriBidi 0.19.7 (SIMPLE) HarfBuzz-ng 2.6.4 (COMPLEX)
[ass] Using font provider fontconfig
[00007f29603cf3d0] avcodec decoder: Using NVIDIA VDPAU Driver Shared Library  460.73.01  Thu Apr  1 21:30:56 UTC 2021 for hardware decoding
[ass] fontselect: (DejaVu Sans, 400, 0) -> /usr/share/fonts/truetype/dejavu/DejaVuSans.ttf, 0, DejaVuSans
[ass] Shaper: FriBidi 0.19.7 (SIMPLE) HarfBuzz-ng 2.6.4 (COMPLEX)
[ass] Using font provider fontconfig
[00007f29601f4ef0] 

In [81]:
import datetime
def sec2hmsms(s):
    td = datetime.timedelta(seconds=s)
    h,m,s = str(td).split(':')
    return '%d:%d:%.2f' % (int(h), int(m), float(s))
#    if len(ret) == 7:
#        return ret + '.00'
#    assert len(ret) == 14, ret
#    return ret[:-4]
sec2hmsms(3600+66.6), sec2hmsms(.06), sec2hmsms(.0006), sec2hmsms(1.)

('1:1:6.60', '0:0:0.06', '0:0:0.00', '0:0:1.00')

In [28]:
str(datetime.timedelta(seconds=.6))

'0:00:00.600000'

In [34]:
out

'\n[Script Info]\nTitle:\nOriginal Script:\nOriginal Translation:\nOriginal Editing:\nOriginal Timing:\nOriginal Script Checking:\nScriptType: v4.00+\nCollisions: Normal\nPlayResY: 404\nPlayDepth: 0\nTimer: 100,0000\n[V4+ Styles]\nFormat: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding\nStyle: subtitle,DejaVu Sans,80,&H00FFFFFF,&H00B4FCFC,&H00000000,&H00000000,0,0,0,0,100,100,0.00,0.00,1,2.50,0.00,2,50,50,50,0\nStyle: subtit25,DejaVu Sans,25,&H00FFFFFF,&H00B4FCFC,&H00000000,&H00000000,0,0,0,0,100,100,0.00,0.00,1,2.50,0.00,2,50,50,50,0\nStyle: concepts,DejaVu Sans,80,&H0000C000,&H00B4FCFC,&H00000000,&H00000000,0,0,0,0,100,100,0.00,0.00,1,2.50,0.00,7,50,50,50,0\nStyle: personte,DejaVu Sans,80,&H00C00000,&H00B4FCFC,&H00000000,&H00000000,0,0,0,0,100,100,0.00,0.00,1,2.50,0.00,6,50,50,50,0\nStyle: personfa,DejaVu